In [1]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
You should consider upgrading via the '/Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/venv/bin/python3.10 -m pip install --upgrade pip' command.


In [4]:
!pip install --upgrade pip

  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [8]:
!git clone https://github.com/deepset-ai/haystack.git

Cloning into 'haystack'...
remote: Enumerating objects: 32720, done.
remote: Counting objects: 100% (620/620), done.
remote: Compressing objects: 100% (287/287), done.
eiving objects:  20% (6544/32720), 21.05 MiB | 1.03 MiB/sReceiving objects:  23% (7526/32720), 21.69 MiB | 1.08 MiB/sReceiving objects:  25% (8180/32720), 21.69 MiB | 1.08 MiB/sReceiving objects:  26% (8508/32720), 22.38 MiB | 1.14 MiB/sReceiving objects:  28% (9162/32720), 22.38 MiB | 1.14 MiB/sReceiving objects:  30% (9816/32720), 22.38 MiB | 1.14 MiB/sReceiving objects:  32% (10471/32720), 23.11 MiB | 1.20 MiB/sReceiving objects:  33% (10830/32720), 23.91 MiB | 1.28 MiB/sReceiving objects:  35% (11452/32720), 23.91 MiB | 1.28 MiB/sReceiving objects:  37% (12107/32720), 24.73 MiB | 1.32 MiB/sReceiving objects:  40% (13088/32720), 24.73 MiB | 1.32 MiB/sReceiving objects:  41% (13416/32720), 26.61 MiB | 1.48 MiB/sReceiving objects:  42% (13743/32720), 27.46 MiB | 1.54 MiB/sReceiving objects:  44% (14397/32720), 27.46 MiB

In [18]:
!brew install postgresql
!brew install cmake
!brew install rust

In [5]:
!python --version

Python 3.10.2


In [17]:
!cd haystack && pip install -e .

In [1]:
from pprint import pprint
from tqdm.auto import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

In [21]:
import os
import subprocess

es_server = subprocess.Popen(
    ["/Users/khumoyunaminaddinov/Downloads/elasticsearch-7.9.2/bin/elasticsearch"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)
# wait until ES has started
! sleep 30

In [23]:
es_server.communicate()

(b'[2023-03-21T10:52:59,130][INFO ][o.e.n.Node               ] [Khumoyuns-MacBook-Air.local] version[7.9.2], pid[651], build[default/tar/d34da0ea4a966c4e49417f2da2f244e3e97b4e6e/2020-09-23T00:45:33.626720Z], OS[Mac OS X/11.2/x86_64], JVM[AdoptOpenJDK/OpenJDK 64-Bit Server VM/16.0.1/16.0.1+9]\n[2023-03-21T10:52:59,134][INFO ][o.e.n.Node               ] [Khumoyuns-MacBook-Air.local] JVM home [/Library/Java/JavaVirtualMachines/adoptopenjdk-16.jdk/Contents/Home]\n[2023-03-21T10:52:59,135][INFO ][o.e.n.Node               ] [Khumoyuns-MacBook-Air.local] JVM arguments [-Xshare:auto, -Des.networkaddress.cache.ttl=60, -Des.networkaddress.cache.negative.ttl=10, -XX:+AlwaysPreTouch, -Xss1m, -Djava.awt.headless=true, -Dfile.encoding=UTF-8, -Djna.nosys=true, -XX:-OmitStackTraceInFastThrow, -XX:+ShowCodeDetailsInExceptionMessages, -Dio.netty.noUnsafe=true, -Dio.netty.noKeySetOptimization=true, -Dio.netty.recycler.maxCapacityPerThread=0, -Dio.netty.allocator.numDirectArenas=0, -Dlog4j.shutdownHookEna

In [9]:
import re

def extract_text(d, text_list):
    if isinstance(d, dict):
        if "document" in d and isinstance(d["document"], dict):
            extract_text(d["document"], text_list)

        if "section" in d and isinstance(d["section"], list):
            extract_text(d["section"], text_list)
        if "section" in d and isinstance(d["section"], dict):
            extract_text(d["section"], text_list)

        if "paragraph" in d and isinstance(d["paragraph"], list):
            extract_text(d["paragraph"], text_list)
        if "paragraph" in d and isinstance(d["paragraph"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["paragraph"].strip()))
        if "paragraph" in d and isinstance(d["paragraph"], dict):
            extract_text(d["paragraph"], text_list)

        if "@names" in d and isinstance(d["@names"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["@names"].strip()))

        if "#text" in d and isinstance(d["#text"], str):
            text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d["#text"].strip()))
        if "desc_content" in d and isinstance(d["desc_content"], dict):
            extract_text(d["desc_content"], text_list)
        if "desc_name" in d and isinstance(d["desc_name"], dict):
            extract_text(d["desc_name"], text_list)
#         if "literal_block" in d and isinstance(d["literal_block"], list):
#             extract_text(d["literal_block"], text_list)
        if "desc_signature" in d and isinstance(d["desc_signature"], dict):
            extract_text(d["desc_signature"], text_list)
        if "desc" in d and isinstance(d["desc"], list):
            extract_text(d["desc"], text_list)
        if "desc_content" in d and isinstance(d["desc_content"], dict):
            extract_text(d["desc_content"], text_list)

    elif isinstance(d, list):
        for item in d:
            extract_text(item, text_list)
    elif isinstance(d, str):
        text_list.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', d.strip()))

In [18]:
import json

collections_json = json.loads(open('../data/collections.json', 'r').read())
document_texts = []
extract_text(collections_json, document_texts)
open('output.txt', 'w').write(str(document_texts))
len(document_texts)

96

In [25]:
# Option 1: Start Elasticsearch service via Docker
launch_es()

In [38]:
docs = [{"content": x} for x in document_texts]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

/Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/venv/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Using sep_token, but it is not set yet.


In [39]:
questions = []
answers = []

In [40]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(document_store):

    # print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = qag_pipeline.run(documents=[document])
    # print_questions(result)

    for i, ans in enumerate(result['answers']):
        if ans:
            answers.append(ans)
            questions.append(result['queries'][i])

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

...

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [41]:
len(questions), len(answers)

(141, 141)

### Train DecoderGPT